## S3 Python API Example Code

#### Boto3 is used as the S3 API module

In [1]:
import json
import boto3
import boto3.session
import random
import string

#### Set the endpoint and generate a unique bucket name

In [2]:
endpoint = 'https://webscaledemo.netapp.com:8082'
bucket_name = 'tmp-bucket-' + ''.join([random.choice(string.ascii_lowercase) for i in range(4)])
print("Using bucket: " + bucket_name)

Using bucket: tmp-bucket-lbmg


#### Provide your own access credentials

In [3]:
access_key = '<your access key here>'
secret_key = '<your secret access key here>'

**Do not use the example below in production - disabling SSL verification is discouraged!**  
When using a self-signed certificate, make sure to pass it into the constructor:  
```s3 = session.resource(service_name='s3', endpoint_url=endpoint, verify='server_cert.pem')```

In [4]:
session = boto3.session.Session(aws_access_key_id=access_key, aws_secret_access_key=secret_key)
s3 = session.resource(service_name='s3', endpoint_url=endpoint, verify=False)
client = s3.meta.client

### Bucket related operations

#### Create new bucket for S3 account

In [5]:
bucket = s3.Bucket(bucket_name)
bucket.create()

{'Location': '/tmp-bucket-lbmg',
 'ResponseMetadata': {'HTTPHeaders': {'connection': 'KEEP-ALIVE',
   'content-length': '0',
   'date': 'Tue, 21 Mar 2017 17:45:57 GMT',
   'location': '/tmp-bucket-lbmg',
   'server': 'StorageGRID/10.3.0.1',
   'x-amz-request-id': '714493066'},
  'HTTPStatusCode': 200,
  'HostId': '',
  'RequestId': '714493066',
  'RetryAttempts': 0}}

#### List all buckets for S3 account

In [6]:
bucket.wait_until_exists()
for bucket in s3.buckets.all():
    print(bucket.name)

tmp-bucket-lbmg


#### Enable bucket versioning

In [7]:
bucket.Versioning().enable()

{'ResponseMetadata': {'HTTPHeaders': {'connection': 'KEEP-ALIVE',
   'content-length': '0',
   'date': 'Tue, 21 Mar 2017 17:46:07 GMT',
   'server': 'StorageGRID/10.3.0.1',
   'x-amz-request-id': '714493066'},
  'HTTPStatusCode': 200,
  'HostId': '',
  'RequestId': '714493066',
  'RetryAttempts': 0}}

#### Get Bucket Versioning status

In [8]:
print (bucket.Versioning().status)

Enabled


#### Create a bucket policy  
*Note the StorageGRID Webscale specific Resource URN*

In [9]:
policy = {
    "Statement": [
        {
            "Sid":"AddPerm",
            "Effect":"Allow",
            "Principal": "*",
            "Action":["s3:ListBucket"],
            "Resource":["urn:sgws:s3:::" + bucket.name]
        }
    ]
}

Set a new policy allowing anyone to list the contents of the bucket

In [10]:
bucket.Policy().put(Policy=json.dumps(policy))

{'ResponseMetadata': {'HTTPHeaders': {'connection': 'KEEP-ALIVE',
   'content-length': '0',
   'date': 'Tue, 21 Mar 2017 17:46:08 GMT',
   'server': 'StorageGRID/10.3.0.1',
   'x-amz-request-id': '714493066'},
  'HTTPStatusCode': 204,
  'HostId': '',
  'RequestId': '714493066',
  'RetryAttempts': 0}}

### Object related operations

#### Put a new object to a bucket

In [11]:
obj = s3.Object(bucket.name, 'my-key')
obj.put(Body='This is my object\'s data',
        Metadata={'customerid': '1234', 'location': 'germany'},
        ServerSideEncryption='AES256')

{'ETag': '"812d5775a52f93fca96d0ff77f0f5dd0"',
 'ResponseMetadata': {'HTTPHeaders': {'connection': 'KEEP-ALIVE',
   'content-length': '0',
   'date': 'Tue, 21 Mar 2017 17:46:08 GMT',
   'etag': '"812d5775a52f93fca96d0ff77f0f5dd0"',
   'server': 'StorageGRID/10.3.0.1',
   'x-amz-request-id': '714493066',
   'x-amz-server-side-encryption': 'AES256',
   'x-amz-version-id': 'NDM5OEI0NzgtMEU1RS0xMUU3LTg4MzAtQUI2OTAwQzMyQkUx'},
  'HTTPStatusCode': 200,
  'HostId': '',
  'RequestId': '714493066',
  'RetryAttempts': 0},
 'ServerSideEncryption': 'AES256',
 'VersionId': 'NDM5OEI0NzgtMEU1RS0xMUU3LTg4MzAtQUI2OTAwQzMyQkUx'}

#### Copy an existing object

In [12]:
obj.wait_until_exists()
copied_obj = s3.Object(bucket.name, 'my-copied-key')
copied_obj.copy_from(CopySource={ 'Bucket': obj.bucket_name, 'Key': obj.key })

{'CopyObjectResult': {'ETag': '"812d5775a52f93fca96d0ff77f0f5dd0"',
  'LastModified': datetime.datetime(2017, 3, 21, 17, 46, 8, 551000, tzinfo=tzutc())},
 'CopySourceVersionId': 'NDM5OEI0NzgtMEU1RS0xMUU3LTg4MzAtQUI2OTAwQzMyQkUx',
 'ResponseMetadata': {'HTTPHeaders': {'connection': 'CLOSE',
   'content-type': 'application/xml',
   'date': 'Tue, 21 Mar 2017 17:46:08 GMT',
   'server': 'StorageGRID/10.3.0.1',
   'x-amz-copy-source-version-id': 'NDM5OEI0NzgtMEU1RS0xMUU3LTg4MzAtQUI2OTAwQzMyQkUx',
   'x-amz-request-id': '714493066',
   'x-amz-version-id': 'NDNDRjQ2NjctMEU1RS0xMUU3LThBNTQtMkI0NjAwQzMyQkUx'},
  'HTTPStatusCode': 200,
  'HostId': '',
  'RequestId': '714493066',
  'RetryAttempts': 0},
 'VersionId': 'NDNDRjQ2NjctMEU1RS0xMUU3LThBNTQtMkI0NjAwQzMyQkUx'}

#### Get object from bucket

In [13]:
response = obj.get()
data = response['Body'].read()
metadata = response['Metadata']
print("Data: %s // Metadata: %s" % (data, metadata))

Data: b"This is my object's data" // Metadata: {'customerid': '1234', 'location': 'germany'}


#### List all objects for a bucket

In [14]:
for obj in bucket.objects.all():
    print(obj.key)

my-copied-key
my-key


#### Generate a pre-signed URL (only possible via client, not directly via Object object)

In [15]:
url = client.generate_presigned_url('get_object', {'Bucket': bucket.name, 'Key': 'my-key'}, ExpiresIn=3600)
print("Pre-signed URL: %s" % (url))

Pre-signed URL: https://webscaledemo.netapp.com:8082/tmp-bucket-lbmg/my-key?Expires=1490121968&Signature=vEFZlMkrfP2ezsQDd%2BM7MFC7pzs%3D&AWSAccessKeyId=L68PIKZBX88HCUSC41XR


#### Delete all objects and versions from the bucket

In [16]:
for obj in bucket.object_versions.all():
    obj.delete()

#### Delete bucket

In [17]:
bucket.delete()

{'ResponseMetadata': {'HTTPHeaders': {'connection': 'KEEP-ALIVE',
   'content-length': '0',
   'date': 'Tue, 21 Mar 2017 17:46:19 GMT',
   'server': 'StorageGRID/10.3.0.1',
   'x-amz-request-id': '2711551009'},
  'HTTPStatusCode': 204,
  'HostId': '',
  'RequestId': '2711551009',
  'RetryAttempts': 0}}